In [ ]:
%matplotlib inline
import torch
use_cuda = torch.cuda.is_available()
#CUDA_VISIBLE_DEVICES=0,4,8

from numpy.random import RandomState
import numpy as np
torch.manual_seed(42)
prng = RandomState(42)

from torchvision import datasets, transforms
import sys
import os
import csv

if use_cuda:
    num_workers = 4
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False
    
def get_dir(name):
    base_dir = sys.path[0]
    full_path = os.path.join(base_dir, name)
    if not os.path.exists(full_path):
        os.makedirs(full_path)
    return full_path

def train_data(data_name='MNIST',num_data=400):
    train_data = datasets.MNIST(get_dir(data_name),
                train=True, download=True,
                transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                ]))

    #Extract a subset of 5000 samples from MNIST training
    random_permute=prng.permutation(np.arange(0,60000))[0:num_data]
    train_data.train_data = train_data.train_data[random_permute]
    train_data.train_labels = train_data.train_labels[random_permute]
    train_loader = torch.utils.data.DataLoader(train_data,
        batch_size=128, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
    print(len(train_loader))
    return train_loader

def test_data(data_name='MNIST'):
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            get_dir('data_name'),
            train=False, download=True,transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
    ])),
        batch_size=128, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
    #print(len(test_loader))
    return test_loader




In [ ]:
import matplotlib.pyplot as plt
from sklearn import manifold
def feature_extraction(train_loader, scattering):
    print("feature extraction begin~~~")
    features = []
    datas = []
    labels = []
    for i,(data,label) in enumerate(train_loader):
        datas.append(data)
        labels.append(label)
        data, label = data.cuda(), label.cuda()
        feature = scattering(data)
        features.append(feature) 
    print("feature extraction finish~~!")
    return datas,labels,features

def data_visilization(datas,targets):
    '''显示原始数据'''
    print("visilize data~~")
    n = 20  # 每行20个数字，每列20个数字
    img = np.zeros((30 * n, 30 * n))
    for i in range(n):
        ix = 30 * i + 1
        for j in range(n):
            iy = 30 * j + 1
            img[ix:ix + 28, iy:iy + 28] = datas[i * n + j].reshape((28, 28))
    plt.figure(figsize=(28, 28))
    plt.imshow(img, cmap=plt.cm.binary)
    plt.xticks([])
    plt.yticks([])
    plt.show()

def feature_visilization(features,labels):

    '''嵌入空间可视化'''
    print("feature visilize ~~~")
    f = []
    target = []
    for j in range(len(features)):
        for i in features[j]:
            f.append(i.reshape(-1).tolist())
    features = f
    for i in range(len(labels)):
        for j in labels[i]:
            target.append(j.tolist())
    labels = target
    #features = [i for i in f]
    #print(features[0].shape)
    tsne = manifold.TSNE(n_components=2, init='pca', random_state=501)
    X_tsne = tsne.fit_transform(features)
    #print("Org data dimension is {}.Embedded data dimension is {}".format(features.shape[-1], X_tsne.shape[-1]))

    x_min, x_max = X_tsne.min(0), X_tsne.max(0)
    X_norm = (X_tsne - x_min) / (x_max - x_min)  # 归一化
    plt.figure(figsize=(28, 28))
    for i in range(X_norm.shape[0]):
        #print(type(X_norm[i,0]))
        #print(X_norm[i,0])
        plt.text(float(X_norm[i, 0]), float(X_norm[i, 1]), str(labels[i]), color=plt.cm.Set1(int(labels[i])), 
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [ ]:
import torch.nn as nn
class View(nn.Module):
    def __init__(self, *args):
        super(View, self).__init__()
        self.shape = args

    def forward(self, x):
        return x.view(-1,*self.shape)

import torch.nn.functional as F

def train(model, labels,features, optimizer):
    model.train()
    for i in range(len(labels)):
        target = labels[i].cuda()
        optimizer.zero_grad()
        #print(features[i].shape)
        output = model(features[i])
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()


def test(model, test_loader, scattering):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(),target.cuda()
            output = model(scattering(data))
            pred = output.max(1, keepdim = True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    return 100. * correct / len(test_loader.dataset)

In [1]:
if __name__=="__main__":
    from scattering2d import Scattering2D
    import torch.optim
    import torch.nn as nn
    scattering = Scattering2D(shape = (28, 28), J=2)
    scattering = scattering.cuda()
    train_loader = train_data(data_name='MNIST',num_data=5000)
    datas,labels,features = feature_extraction(train_loader,scattering)
    print(features[0].shape)
    #data_visilization(datas,labels)
    feature_visilization(features,labels)
    K=81
    
    model = nn.Sequential(
    View(K, 7, 7),
    nn.BatchNorm2d(K),
    View(K * 7 * 7),
    nn.Linear(K * 7 * 7, 10)
).cuda()

    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9,
                            weight_decay=0.0005)
    for epoch in range(0, 20):
        train(model, labels,features, optimizer)
                   
    test_loader = test_data(data_name='MNIST')
    acc = test(model, test_loader, scattering)
    print('Scattering order  2 linear model test accuracy: %.2f'%(acc) )
    

ModuleNotFoundError: No module named 'torch'